# Intro. to Snorkel: Extracting Spouse Relations from the News

## Part III: Training an End Extraction Model

In this final section of the tutorial, we'll use the noisy training labels we generated in the last tutorial part to train our end extraction model.

For this tutorial, we will be training a Bi-LSTM, a state-of-the-art deep neural network implemented in [TensorFlow](https://www.tensorflow.org/).

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

# TO USE A DATABASE OTHER THAN SQLITE, USE THIS LINE
# Note that this is necessary for parallel execution amongst other things...
# os.environ['SNORKELDB'] = 'postgres:///snorkel-intro'

from snorkel import SnorkelSession
session = SnorkelSession()

We repeat our definition of the `Spouse` `Candidate` subclass:

In [2]:
from snorkel.models import candidate_subclass

Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

We reload the probabilistic training labels:

In [3]:
from snorkel.annotations import load_marginals

train_marginals = load_marginals(session, split=0)

We also reload the candidates:

In [4]:
train_cands = session.query(Spouse).filter(Spouse.split == 0).order_by(Spouse.id).all()
dev_cands   = session.query(Spouse).filter(Spouse.split == 1).order_by(Spouse.id).all()
test_cands  = session.query(Spouse).filter(Spouse.split == 2).order_by(Spouse.id).all()

Finally, we load gold labels for evaluation:

In [28]:
L_gold_dev.shape

(2811, 1)

In [5]:
from snorkel.annotations import load_gold_labels

L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)

Now we can setup our discriminative model. Here we specify the model and learning hyperparameters.

They can also be set automatically using a search based on the dev set with a [GridSearch](https://github.com/HazyResearch/snorkel/blob/master/snorkel/learning/utils.py) object.

In [6]:
from snorkel.learning.pytorch import LSTM

train_kwargs = {
    'lr':            0.01,
    'embedding_dim': 50,
    'hidden_dim':    50,
    'n_epochs':      20,
    'dropout':       0.25,
    'seed':          1701
}

lstm = LSTM(n_threads=None)
lstm.train(train_cands, train_marginals, X_dev=dev_cands, Y_dev=L_gold_dev, **train_kwargs)

[LSTM] Training model
[LSTM] n_train=17241  #epochs=20  batch size=64


/home/cperreault/anaconda3/envs/snorkel-extraction/lib/python3.6/site-packages/torch/nn/functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[LSTM] Epoch 1 (98.96s)	Average loss=0.621418	Dev F1=34.52
[LSTM] Epoch 2 (205.12s)	Average loss=0.602544	Dev F1=38.19
[LSTM] Epoch 3 (310.52s)	Average loss=0.600012	Dev F1=38.31
[LSTM] Epoch 4 (417.26s)	Average loss=0.596670	Dev F1=38.80
[LSTM] Epoch 5 (526.76s)	Average loss=0.593958	Dev F1=37.34
[LSTM] Epoch 6 (632.02s)	Average loss=0.591960	Dev F1=39.25
[LSTM] Epoch 7 (737.44s)	Average loss=0.590146	Dev F1=37.79
[LSTM] Epoch 8 (843.75s)	Average loss=0.589350	Dev F1=39.82
[LSTM] Epoch 9 (950.00s)	Average loss=0.588057	Dev F1=35.49
[LSTM] Epoch 10 (1057.21s)	Average loss=0.587207	Dev F1=36.24
[LSTM] Epoch 11 (1169.07s)	Average loss=0.586520	Dev F1=36.65
[LSTM] Epoch 12 (1275.68s)	Average loss=0.585722	Dev F1=41.35
[LSTM] Epoch 13 (1382.80s)	Average loss=0.584828	Dev F1=38.24
[LSTM] Epoch 14 (1530.79s)	Average loss=0.584799	Dev F1=36.74
[LSTM] Epoch 15 (1737.80s)	Average loss=0.584585	Dev F1=38.76
[LSTM] Epoch 16 (1928.09s)	Average loss=0.583951	Dev F1=36.49
[LSTM] Epoch 17 (2122.83s)	

Now, we get the precision, recall, and F1 score from the discriminative model:

In [7]:
p, r, f1 = lstm.score(test_cands, L_gold_test)
print("Prec: {0:.3f}, Recall: {1:.3f}, F1 Score: {2:.3f}".format(p, r, f1))

Prec: 0.307, Recall: 0.706, F1 Score: 0.428


We can also get the candidates returned in sets (true positives, false positives, true negatives, false negatives) as well as a more detailed score report:

In [8]:
tp, fp, tn, fn = lstm.error_analysis(session, test_cands, L_gold_test)

Scores (Un-adjusted)
Pos. class accuracy: 0.706
Neg. class accuracy: 0.86
Precision            0.307
Recall               0.706
F1                   0.428
----------------------------------------
TP: 154 | FP: 348 | TN: 2135 | FN: 64



Note that if this is the final test set that you will be reporting final numbers on, to avoid biasing results you should not inspect results.  However you can run the model on your _development set_ and, as we did in the previous part with the generative labeling function model, inspect examples to do error analysis.

You can also improve performance substantially by increasing the number of training epochs!

Finally, we can save the predictions of the model on the test set back to the database. (This also works for other candidate sets, such as unlabeled candidates.)

In [29]:
lstm.save('weak_supervision')

[LSTM] Model saved as <weak_supervision>


In [9]:
lstm.save_marginals(session, test_cands)

Saved 2701 marginals


In [32]:
lstm_small = LSTM(n_threads=None)

In [45]:
labels = L_gold_dev.toarray()

In [46]:
labels[labels != 1] = 0

In [49]:
train_marginals.shape

(22254,)

In [50]:
lstm_small.train(dev_cands, labels.reshape(2811,), **train_kwargs)

[LSTM] Training model
[LSTM] n_train=2811  #epochs=20  batch size=64
[LSTM] Epoch 1 (28.14s)	Average loss=0.300984
[LSTM] Epoch 2 (56.03s)	Average loss=0.248689
[LSTM] Epoch 3 (85.21s)	Average loss=0.248711
[LSTM] Epoch 4 (113.31s)	Average loss=0.245092
[LSTM] Epoch 5 (141.55s)	Average loss=0.217695
[LSTM] Epoch 6 (168.59s)	Average loss=0.167240
[LSTM] Epoch 7 (199.28s)	Average loss=0.137144
[LSTM] Epoch 8 (227.53s)	Average loss=0.119306
[LSTM] Epoch 9 (255.76s)	Average loss=0.107908
[LSTM] Epoch 10 (282.96s)	Average loss=0.101803
[LSTM] Epoch 11 (311.17s)	Average loss=0.094658
[LSTM] Epoch 12 (340.20s)	Average loss=0.088760
[LSTM] Epoch 13 (368.99s)	Average loss=0.089042
[LSTM] Epoch 14 (397.25s)	Average loss=0.084394
[LSTM] Epoch 15 (423.96s)	Average loss=0.076003
[LSTM] Epoch 16 (453.84s)	Average loss=0.076825
[LSTM] Epoch 17 (481.91s)	Average loss=0.076285
[LSTM] Epoch 18 (510.62s)	Average loss=0.068815
[LSTM] Epoch 19 (539.43s)	Average loss=0.071791
[LSTM] Epoch 20 (569.40s)	Avera

In [51]:
lstm_small.save('small_dataset')

[LSTM] Model saved as <small_dataset>


In [52]:
tp, fp, tn, fn = lstm_small.error_analysis(session, test_cands, L_gold_test)

Scores (Un-adjusted)
Pos. class accuracy: 0.165
Neg. class accuracy: 0.922
Precision            0.157
Recall               0.165
F1                   0.161
----------------------------------------
TP: 36 | FP: 194 | TN: 2289 | FN: 182



##### More importantly, you completed the introduction to Snorkel! Give yourself a pat on the back!

In [14]:
y = L_gold_test.toarray()

In [21]:
q =L_gold_dev.toarray()

In [35]:
train_marginals[:10]

array([ 0.19843132,  0.19843132,  0.19843132,  0.19843132,  0.5       ,
        0.5       ,  0.19843132,  0.19843132,  0.5       ,  0.19843132])

Spouse(Span("b'Saif'", sentence=14862, chars=[224,227], words=[41,41]), Span("b" Ileana D'Cruz"", sentence=14862, chars=[308,321], words=[60,62]))